# 데이터 분석 순서(KDD 분석 방법론)
* 데이터 세트 선택: CSV,EXCEL, DB에서 데이터를 읽어옴
* 데이터 전처리: 데이터타입, 결측값, 이상치탐지, 데이터분포분석, 상관관계
* 데이터 변환(특성추출) : 원본데이터에서 새로운 데이터 생성, 삭제, 스케일링, 구간화
* 데이터 마이닝(모델만들기, 분석): 분석에 적합한 알고리즘 선택, 모델생성, 튜닝
* 결과 평가 : 테스트 데이터를 이용해서 데이터 마이닝으로 만든모델의 성능 평가

# 데이터 전처리
* 데이터 타입 변환
* 결측치 탐지 및 보간
* 이상치 탐지 및 처리
* 데이터 특성 파악(치우침, 분포 특성)
* 변수들 간의 상관관계 분석

In [1]:
import pandas as pd
import numpy as np

# 1. 데이터 세트 선택 및 로딩
* 데이터 로드 후 head(),tail()로 컬럼과 데이터 파악

In [130]:
data = pd.read_csv("./data/Titanic_train.csv")
data.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


# 2. .info()로 컬럼명, 결측치, 데이터 파악

In [5]:
# 데이터가 너무 커서 Non-null이 표시 안될떄 : show_counts=True
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


# 3. describe()로 기초통계량 파악(이상치 파악)

In [6]:
data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


# 4. 결측값 찾고, 비율보고, 대치/삭제하기
* 결측값 비율 계산: isna().sum() / len(데이터프레임) * 100
* 결측값 비율이 5% 미만: 행, 열을 제거, 분석에 크게 영향을 미치지 않음 
* 결측값 비율이 5%~ 30%: 결측값을 대체(inputition)
   * 수치형 데이터 (숫자형, 나이, 가격):평균(mean), 중앙값(median),최빈값(mode)으로 대체
   * 범주형 데이터(숫자형/문자형, 선실등급, 탑승지:최빈값(mode)으로 대체
* 결측값 비율이 30~ 50%: 컬럼의 중요도에 따라서 결측값을 대체 혹은 삭제
   * KNN(K-Nearest Neighbor, 최근접이웃) imputer, 회귀분석을 통해 결측값 대체
* 결측값 비율이 50% 이상: 해당 컬럼 삭제

In [9]:
round(data.isna().sum()/len(data)*100,2)

PassengerId     0.00
Survived        0.00
Pclass          0.00
Name            0.00
Sex             0.00
Age            19.87
SibSp           0.00
Parch           0.00
Ticket          0.00
Fare            0.00
Cabin          77.10
Embarked        0.22
dtype: float64

In [10]:
data['Age'].mode()

0    24.0
Name: Age, dtype: float64

## 결측 데이터 종류
* MCAR(완전 무작위 결측): 다른 변수와 무관하게 생긴 결측 (랜덤, 이유없음)
   * 설문조사중 일부 응답자가 무작위로 답변을 건너뜀.
* MAR(무작위 결측): 다른 변수와는 관련 있지만 본인과는 무관
   * 나이가 많은 사람들이 소득란에 응답을 하지 않는 경우(나이와 관련)
   * 직업에 따라서 학력란에 응답하지 않는 경우(가정환경, 소득)
* NMAR(비무작위 결측): 변수 자기 자신과 직접 관련이 있는 경우
   * 소득이 낮은 사람이 자신의 소득을 숨기는 경우(소득과 결측이 직접 관련)
   * 체중이 많이 나가는 경우 자신의 체중을 숨기는 경우
   * 만족도가 낮은 고객이 만족도 조사에 응하지 않는 경우

## 결측치 처리방법
* 1)단순대치법(simple imputation)
  * (1) 완전 분석: 결측값이 있는 모든 행을 삭제하고 완전한 자료만으로 분석 (잘 안씀)
     * 결측값을 삭제해도 모델을 만들기에 충분히 많은 데이터가 있는 경우
     * 결측값을 삭제한 후에 데이터의 편향이 없다는 전제가 있을 때
     * dropna() : 결측값 삭제

In [13]:
data.dropan()

AttributeError: 'DataFrame' object has no attribute 'dropan'

   * (2) 평균대치법 : 결측치가 있는 컬럼에서 데이터의 평균을 구한 후 결측값을 대치
   * 평균을 이용하기 떄문에 간편
   * 데이터에 이상치가 있을 경우 평균을 이용할 수 없다. 
   * 데이터에 이상치가 있는 경우 중앙값이나 최빈값을 고려해야한다.

In [24]:
a = np.array([24,5,10,34,20,18,28,23])
b = np.array([24,5,10,34,20,18,28, 2000])

In [15]:
a.mean()

np.float64(19.875)

In [16]:
b.mean()

np.float64(239.88888888888889)

In [25]:
np.median(a)

np.float64(21.5)

In [26]:
np.median(b)

np.float64(22.0)

Age 컬럼의 결측값을 평균 대치법으로 대치

In [31]:
data['Age'].isna().sum()

np.int64(177)

In [29]:
data['Age'].describe()
# mean값의 세배정동에 max값이 있으면 이상치의 데이터라고 생각하면 된다. 
# 평균과 중앙값이 비슷하면 이상치가 크게 두드러지지 않음이라고 이해하면 됨.

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [35]:
age_an_idx = data[data4['Age'].isna()].index

In [96]:
# 깊은 복사, 얕은 복사
data2 = data.copy()
data3 = data.copy()
data4 = data.copy()

In [59]:
data4['Age'] = data4['Age'].fillna(data4['Age'].mean())

In [60]:
data4.loc[age_an_idx]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
5,6,0,3,"Moran, Mr. James",male,29.699118,0,0,330877,8.4583,NaN,Q
17,18,1,2,"Williams, Mr. Charles Eugene",male,29.699118,0,0,244373,13.0000,NaN,S
19,20,1,3,"Masselmani, Mrs. Fatima",female,29.699118,0,0,2649,7.2250,NaN,C
26,27,0,3,"Emir, Mr. Farred Chehab",male,29.699118,0,0,2631,7.2250,NaN,C
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,29.699118,0,0,330959,7.8792,NaN,Q
...,...,...,...,...,...,...,...,...,...,...,...,...
859,860,0,3,"Razi, Mr. Raihed",male,29.699118,0,0,2629,7.2292,NaN,C
863,864,0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,29.699118,8,2,CA. 2343,69.5500,NaN,S
868,869,0,3,"van Melkebeke, Mr. Philemon",male,29.699118,0,0,345777,9.5000,NaN,S
878,879,0,3,"Laleff, Mr. Kristo",male,29.699118,0,0,349217,7.8958,NaN,S


중앙값으로 대치

In [42]:
data2['Age'].median()

np.float64(28.0)

In [41]:
age_an_idx2 = data2[data2['Age'].isna()].index

In [43]:
data2['Age'] = data2['Age'].fillna(data2['Age'].median())

In [44]:
data2.loc[age_an_idx2]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
5,6,0,3,"Moran, Mr. James",male,28.0,0,0,330877,8.4583,NaN,Q
17,18,1,2,"Williams, Mr. Charles Eugene",male,28.0,0,0,244373,13.0000,NaN,S
19,20,1,3,"Masselmani, Mrs. Fatima",female,28.0,0,0,2649,7.2250,NaN,C
26,27,0,3,"Emir, Mr. Farred Chehab",male,28.0,0,0,2631,7.2250,NaN,C
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,28.0,0,0,330959,7.8792,NaN,Q
...,...,...,...,...,...,...,...,...,...,...,...,...
859,860,0,3,"Razi, Mr. Raihed",male,28.0,0,0,2629,7.2292,NaN,C
863,864,0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,28.0,8,2,CA. 2343,69.5500,NaN,S
868,869,0,3,"van Melkebeke, Mr. Philemon",male,28.0,0,0,345777,9.5000,NaN,S
878,879,0,3,"Laleff, Mr. Kristo",male,28.0,0,0,349217,7.8958,NaN,S


최빈값으로 대치

In [51]:
data3['Age'].mode()[0]

np.float64(24.0)

In [48]:
an_idx3 = data2[data3['Age'].isna()].index

In [53]:
data3['Age'] = data3['Age'].fillna(data3['Age'].mode()[0])

In [54]:
data3['Age'].fillna(data3['Age'].mode())

0      22.0
1      38.0
2      26.0
3      35.0
4      35.0
       ... 
886    27.0
887    19.0
888    24.0
889    26.0
890    32.0
Name: Age, Length: 891, dtype: float64

In [55]:
data3.loc[age_an_idx3]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
5,6,0,3,"Moran, Mr. James",male,24.0,0,0,330877,8.4583,NaN,Q
17,18,1,2,"Williams, Mr. Charles Eugene",male,24.0,0,0,244373,13.0000,NaN,S
19,20,1,3,"Masselmani, Mrs. Fatima",female,24.0,0,0,2649,7.2250,NaN,C
26,27,0,3,"Emir, Mr. Farred Chehab",male,24.0,0,0,2631,7.2250,NaN,C
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,24.0,0,0,330959,7.8792,NaN,Q
...,...,...,...,...,...,...,...,...,...,...,...,...
859,860,0,3,"Razi, Mr. Raihed",male,24.0,0,0,2629,7.2292,NaN,C
863,864,0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,24.0,8,2,CA. 2343,69.5500,NaN,S
868,869,0,3,"van Melkebeke, Mr. Philemon",male,24.0,0,0,345777,9.5000,NaN,S
878,879,0,3,"Laleff, Mr. Kristo",male,24.0,0,0,349217,7.8958,NaN,S


# scikit-learn의 simple imputer를 이용한 대치

In [62]:
from sklearn.impute import SimpleImputer

In [ ]:
an_idx= data[data['Age'].isna()].index

In [72]:
imp_mean = SimpleImputer(strategy='mean')
data4['Age'] = imp_mean.fit_transform(data4['Age'].values.reshape(-1,1))[:,0] #차원을 맞춰줘야함

In [68]:
data4['Age'].isna().sum()

np.int64(0)

In [69]:
imp_median = SimpleImputer(strategy='median')
data3['Age'] =imp_median.fit_transform(data3[['Age']])[:,0]

In [70]:
data3.loc[na_idx]

NameError: name 'na_idx' is not defined

In [78]:
imp_mode = SimpleImputer(strategy='most_frequent')
data2['Age'] = imp_mode.fit_transform(data2[['Age']])[:,0]

In [129]:
imp_mode = SimpleImputer(strategy='most_frequent')
fitted = imp_mode.fit(data2[['Age']])
fitted

SimpleImputer(strategy='most_frequent')

In [97]:
data5 = data.copy()

In [98]:
data5.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

* 결측값 비율이 30%-50%일떄 KNN 최근접이웃법

In [100]:
from sklearn.impute import KNNImputer

In [101]:
an_idx= data5[data5['Age'].isna()].index

In [102]:
knn_imp = KNNImputer(n_neighbors=5)
data5['Age'] = knn_imp.fit_transform(data5[['Age']])[:,0]

In [103]:
data5.loc[an_idx]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
5,6,0,3,"Moran, Mr. James",male,29.699118,0,0,330877,8.4583,NaN,Q
17,18,1,2,"Williams, Mr. Charles Eugene",male,29.699118,0,0,244373,13.0000,NaN,S
19,20,1,3,"Masselmani, Mrs. Fatima",female,29.699118,0,0,2649,7.2250,NaN,C
26,27,0,3,"Emir, Mr. Farred Chehab",male,29.699118,0,0,2631,7.2250,NaN,C
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,29.699118,0,0,330959,7.8792,NaN,Q
...,...,...,...,...,...,...,...,...,...,...,...,...
859,860,0,3,"Razi, Mr. Raihed",male,29.699118,0,0,2629,7.2292,NaN,C
863,864,0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,29.699118,8,2,CA. 2343,69.5500,NaN,S
868,869,0,3,"van Melkebeke, Mr. Philemon",male,29.699118,0,0,345777,9.5000,NaN,S
878,879,0,3,"Laleff, Mr. Kristo",male,29.699118,0,0,349217,7.8958,NaN,S


# 깊은 복사, 얕은 복사
* 깊은 복사 : 완전 두개 메모리에서 두개의 공간을 쓰는 것 
* 얕은 복사 : 껍데기만 바뀌고 같은 메모리를 쓰는 것
* 메모리 용량 문제 때문에 기본적으로 앝은 복사를 지원한다.

In [104]:
data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [105]:
# data를 copy()를 이용해서 복사
data_copyed = data.copy()

# data를 다른 변수에 재할당
data_reassigned = data

In [106]:
data['Age'].isna().sum()

np.int64(177)

In [107]:
data_reassigned['Age'] = data_reassigned['Age'].fillna(28)

In [108]:
data_reassigned['Age'].isna().sum()

np.int64(0)

In [110]:
data['Age'].isna().sum()

np.int64(0)

# 실제 메모리상의 주소를 출력 id()

In [112]:
print("data의 메모리 주소:",id(data) )
print("data_reassigned 의 메모리 주소:",id(data_reassigned) )
print("data_copyed 의 메모리 주소:",id(data_copyed) )

data의 메모리 주소: 3042826344720
data_reassigned 의 메모리 주소: 3042826344720
data_copyed 의 메모리 주소: 3042788463088


In [114]:
data_copyed.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [115]:
new_df = data_copyed[['Survived', 'Pclass', 'Name']]
print("data_copyed:", id(data_copyed))
print("new_df:", id(new_df))

data_copyed: 3042788463088
new_df: 3042826883568


In [117]:
loc_result = data_copyed.loc[data_copyed['Age'] > 50, ['Name', 'Age']]
slice_result = data_copyed.loc[data_copyed['Age'] > 50][['Name', 'Age']]

In [119]:
loc_result

,Name,Age
6,"McCarthy, Mr. Timothy J",54.0
11,"Bonnell, Miss. Elizabeth",58.0
15,"Hewlett, Mrs. (Mary D Kingcome)",55.0
33,"Wheadon, Mr. Edward H",66.0
54,"Ostby, Mr. Engelhart Cornelius",65.0
...,...,...
820,"Hays, Mrs. Charles Melville (Clara Jennings Gr...",52.0
829,"Stone, Mrs. George Nelson (Martha Evelyn)",62.0
851,"Svensson, Mr. Johan",74.0
857,"Daly, Mr. Peter Denis",51.0


In [118]:
slice_result

,Name,Age
6,"McCarthy, Mr. Timothy J",54.0
11,"Bonnell, Miss. Elizabeth",58.0
15,"Hewlett, Mrs. (Mary D Kingcome)",55.0
33,"Wheadon, Mr. Edward H",66.0
54,"Ostby, Mr. Engelhart Cornelius",65.0
...,...,...
820,"Hays, Mrs. Charles Melville (Clara Jennings Gr...",52.0
829,"Stone, Mrs. George Nelson (Martha Evelyn)",62.0
851,"Svensson, Mr. Johan",74.0
857,"Daly, Mr. Peter Denis",51.0


In [120]:
print("data_copyed:", id(data_copyed))
print("loc_result:", id(loc_result))
print("slice_result", id(slice_result))

data_copyed: 3042788463088
loc_result: 3042826342512
slice_result 3042826886976


In [121]:
data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [123]:
data.drop('Cabin', axis=1, inplace=True)

In [124]:
data.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       2
dtype: int64

# 카테고리 변수의 경우 최빈값 .mode()를 이용해 결측값 처리

In [131]:
cabin_idx = data[data['Embarked'].isna()].index

In [132]:
data['Embarked']= data['Embarked'].fillna(data['Embarked'].mode())[0]

In [133]:
data.loc[cabin_idx]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,S
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,S


In [134]:
data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [126]:
data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Embarked'],
      dtype='object')

In [127]:
data7 = data[['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Embarked']]

In [128]:
print(id(data))
print(id(data7))

3042826344720
3042827286704
